Sample Rate - 100 Hz



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

ImportError: Traceback (most recent call last):
  File "c:\users\charan tanuj\appdata\local\programs\python\python38-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "c:\users\charan tanuj\appdata\local\programs\python\python38-32\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\charan tanuj\appdata\local\programs\python\python38-32\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "c:\users\charan tanuj\appdata\local\programs\python\python38-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_i

In [ ]:
df = pd.read_csv("Hand_raise_to_chest_merged.csv")

# New section

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df.dropna(inplace = True)
print(df.shape)
df.isnull().sum()

In [ ]:
df.describe()

**PCA**

In [ ]:
df.head()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
#df1 = df.drop('time', axis = 1)
df1 = df.drop('Time Stamp', axis = 1)
df_normal = MinMaxScaler().fit_transform(df1)
df_normal = pd.DataFrame(df_normal, columns =df1.columns)


In [ ]:
df_normal.hist(figsize=(20,10), grid = False, layout=(2,4), bins = 30);

We notice that there might be a strong relationship between Y_Acc and Z_acc, X_Orient and Y_Orient, X_Acc and Z_Orient

linear models work on the assumption that the distribution of the independent variable and the target variable are similar. Therefore, knowing about the skewness of data helps us in creating better linear models.

In [ ]:
import numpy as np

nf = df_normal.select_dtypes(include=[np.number]).columns
cf = df_normal.select_dtypes(include=[np.object]).columns

In [ ]:
from scipy import stats
skew_list = stats.skew(df_normal[nf])
skew_list_df = pd.concat([pd.DataFrame(nf,columns=['Features']),pd.DataFrame(skew_list,columns=['Skewness'])],axis = 1)
skew_list_df.sort_values(by='Skewness', ascending = False)

We notice that Z_Acc is moderately skewed follwed my Y_Orient. This skewness in Z_Acc feature may be an outcome of outliers. can analyse it further using scatter plots.

In [ ]:
sns.set()
cols = df1.columns
sns.pairplot(df_normal[cols], height = 2.5)
plt.show();

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df_normal.drop(['Y_Orient'], axis = 1)
y = df_normal['Y_Orient']
X_std = StandardScaler().fit_transform(X)
X_mean = np.mean(X_std, axis=0)
#X_cov = np.cov(X_std.T)
X_cov = (X_std - X_mean).T.dot((X_std - X_mean)) / (X_std.shape[0]-1)

In [ ]:
corr_mat = np.corrcoef(X_std.T)
eigenvalues, eigenvectors = np.linalg.eig(corr_mat)
print('\nEigenvalues \n%s' %eigenvalues)

In [ ]:
pairs = [(np.abs(eigenvalues[i]), eigenvectors[:,i]) for i in range(len(eigenvalues))]
pairs.sort(key = lambda x: x[0], reverse = True)

In [ ]:
sorted_eigenval = []
for i in pairs:
    sorted_eigenval.append(i[0])
print(sorted_eigenval)

In [ ]:
total = sum(eigenvalues)
variance_explained = [(i/total)*100 for i in sorted_eigenval]

In [ ]:
variance_explained

In [ ]:
cum_variance_explained = np.cumsum(variance_explained)
cum_variance_explained

In [ ]:
with plt.style.context('fivethirtyeight'):
    plt.figure(figsize=(8, 6))
    plt.bar(range(5), variance_explained, alpha=0.7, align='center',
            label='individual explained variance')
    plt.step(range(5), cum_variance_explained, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout();

In [ ]:
projection_mat = np.hstack((pairs[0][1].reshape(5,1),
                           pairs[1][1].reshape(5,1),
                           pairs[2][1].reshape(5,1) 
                           ))

In [ ]:
X_new = X_std.dot(projection_mat)
X_new.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.32, random_state = 1)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu", max_iter=1000).fit(X_train , y_train)
y_pred=reg.predict(X_test)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
d1 = pd.read_csv("50hz_merged_hand_rise_to_chest.csv")

In [ ]:
d1.head()

In [ ]:
y_test = d1['Y_Orient']
test = d1.drop(['Y_Orient', 'X_Orient', 'X_Orient'], axis = 1)

In [ ]:

y_pred1 = reg.predict(test)
print("The Score with ", (r2_score(y_pred1, y_test)))

**PCA done**

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
sns.kdeplot(data=df)

In [ ]:
df.plot(kind = 'hist', bins = 100, figsize = (16,8))

In [ ]:
X = df.copy()
X.pop('Time Stamp')
X.pop('Y_Orient')
X.pop('Y_Acc')
X.pop('Z_Acc')
X

In [ ]:
#X_train.plot(kind = 'hist', bins = 100, figsize = (16,8))
sns.kdeplot(data=X)

In [ ]:
y = df[['Y_Orient', 'Y_Acc', 'Z_Acc']]
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.32, random_state = 1)

Normalize data

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
X_trainnormal = MinMaxScaler().fit_transform(X_train)
X_trainnormal = pd.DataFrame(X_trainnormal, columns =X_train.columns)
X_trainnormal.describe()

In [ ]:
X_trainnormal

In [ ]:
sns.distplot(X_trainnormal["X_Acc"])

In [ ]:
sns.kdeplot(data=X_trainnormal)

In [ ]:
X_trainnormal.plot(kind = 'hist', bins = 50)

In [ ]:
X_testnormal = MinMaxScaler().fit_transform(X_test)
X_testnormal = pd.DataFrame(X_testnormal, columns =X_test.columns)
X_testnormal.describe()

In [ ]:
X_testnormal

In [ ]:
sns.kdeplot(data=X_testnormal)

In [ ]:
'''
normalized_X1 = preprocessing.normalize(X_train)
normalized_X1 = pd.DataFrame(normalized_X1, columns=X.columns)
normalized_X1.describe()
sns.kdeplot(data=normalized_X1)
normalized_X1
normalized_X1.plot(kind = 'hist', bins = 100)
sns.set(rc={'figure.figsize':(16,8)})
sns.kdeplot(data=normalized_X1)
'''

Standardzing Data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_trainscaled=sc.fit_transform(X_train)
X_testscaled=sc.fit_transform(X_test)

In [ ]:
X_trainscaled = pd.DataFrame(X_trainscaled, columns=X_train.columns)
X_trainscaled

In [ ]:
X_testscaled = pd.DataFrame(X_testscaled, columns=X_test.columns)
X_testscaled

In [ ]:
sns.kdeplot(data=X_trainscaled)

In [ ]:
sns.kdeplot(data=X_testscaled)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
principalComponents = pca.fit_transform(X_trainnormal)
principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
principalDf

In [ ]:
## MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

In [ ]:

reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu", max_iter=1000).fit(X_trainnormal , y_train)
y_pred=reg.predict(X_testnormal)
print("The Score with ", (r2_score(y_pred, y_test)))


In [ ]:
import pickle
HRC_NN_split = 'final_model.sav'
pickle.dump(reg, open(HRC_NN_split, 'wb'))
loaded_model = pickle.load(open(HRC_NN_split, 'rb'))
y_pred = loaded_model.predict(X_testnormal)
print(r2_score(y_pred, y_test))

In [ ]:
reg1 = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu", max_iter=1000).fit(X_trainscaled, y_train)
y_pred=reg1.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
#import pickle
HRC_NN_split = 'final_model.sav'
pickle.dump(reg1, open(HRC_NN_split, 'wb'))
loaded_model = pickle.load(open(HRC_NN_split, 'rb'))
y_pred = loaded_model.predict(X_testscaled)
print(r2_score(y_pred, y_test))

In [ ]:
reg2 = MLPRegressor(hidden_layer_sizes=(256,128,64),activation="relu", max_iter=900).fit(X_trainscaled, y_train)
y_pred=reg2.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
reg3 = MLPRegressor(hidden_layer_sizes=(256,128,64),activation="relu", max_iter=1000).fit(X_trainscaled, y_train)
y_pred=reg3.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
reg4 = MLPRegressor(hidden_layer_sizes=(64,64,64), activation="tanh", max_iter=1000).fit(X_trainscaled, y_train)
y_pred=reg4.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
reg5 = MLPRegressor(hidden_layer_sizes=(256,128,64), activation="tanh", max_iter=1000).fit(X_trainscaled, y_train)
y_pred=reg5.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))